***Tarun Kumar Alapati***
A20218266

In [ ]:
#import necessary packages
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#####Matrix Initilization and Conductance#####

#***Matrix Initilization and Conductance***

In [ ]:
#This function take file path as the input and reads and conversts it into list of lists of all the nodes
def get_list_words(file):
  f = open(file, "r")
  community_list = f.readlines()
  communities =[]
  for each_line in community_list: # iterates though each line (list)
      communities.append([int(a) for a in each_line.split()])
  f.close()
  return communities

#This function 
def set_factor_matrix(nodes,gt_communities, seed_communities):
   fact_matrix = np.zeros((len(nodes),len(gt_communities)))
   for community in range(len(seed_communities)):
    for node in seed_communities[community]:
      fact_matrix[node][community] = 1
   return fact_matrix

def conductance(fact_matrix,G,seed_communities):
  for a in range(len(fact_matrix)): # Iterating through matrix
    value_minimum = np.inf # Initialized minimum value
    for b in range(len(fact_matrix[0])): # Iterating through each node within the factor matrix
      conduct = nx.conductance(G, (seed_communities[b]+list(G.neighbors(a))+[a])) # Computing the conductance associated with the particular node and their associated neighbors data
      if value_minimum > conduct: # If obtained conductance value is less then the minimum value then we reassign them
        value_minimum = conduct # Now we swap the values obtained
        conduct_minimum = b # This stores the node number having the minimum conductance
      for c in list(G.neighbors(a))+[a]: # In this loop we evaluate if 'u' is locally minimal to 'c' and update it with '1' else keep at as '0'
        fact_matrix[c][conduct_minimum] = 1
  return fact_matrix


In [ ]:
# Read the edge list data and Communities
G = nx.read_edgelist("/content/drive/MyDrive/Big Data Assignments/Project5/YouTube.edgelist", nodetype=int)

gt_communities = get_list_words("/content/drive/MyDrive/Big Data Assignments/Project5/groundtruth_communities.txt")
seed_communities = get_list_words("/content/drive/MyDrive/Big Data Assignments/Project5/20percent_seed_communities.txt")

#Intitilaize Factor Matrix and update community list of 20 percent seeds 
fact_matrix = set_factor_matrix(G.nodes,gt_communities,seed_communities)
#Calculate conductance for rest of the values
fact_matrix = conductance(fact_matrix,G,seed_communities)

In [ ]:
fact_matrix

In [ ]:
#Get neighbor communities
neigh_communities = get_list_words("/content/drive/MyDrive/Big Data Assignments/Project5/neighborhood_seeds.txt")

#Intitilaize Factor Matrix and update community list of neighbour seeds 

fact_matrix_neigh = set_factor_matrix(G.nodes,gt_communities,neigh_communities)

#Calculate conductance for rest of the values of neighbour seeds 

fact_matrix_neigh = conductance(fact_matrix,G,neigh_communities)

In [ ]:
fact_matrix_rand = np.random.rand(len(G.nodes),len(gt_communities)) # Creating a simple factor matrix with random values in range between [0,1]
fact_matrix_rand = np.around(fact_matrix_rand,decimals=4)

In [ ]:
fact_matrix_rand
#Factor Matrix Initialization for Neighborhood Seed Communities

#***Matrix Factorization using BigCLAMV2.0***

In [ ]:
########This function takes number of iterations, Intial factor matrix, Communities list and Graph network and return a factorized factor matrix#####
def bigclam_factor(n,fact_matrix,gt_communities,G):
  percentage_change=0
  for iter in range(n): 
    sum_nodes = fact_matrix.sum(axis=0) 
    for i in range(len(fact_matrix)): 
      neighbors_data = list(G.neighbors(i)) 
      sum_nodes_v = np.zeros(len(gt_communities))
      Delta_1 = np.zeros(len(gt_communities))
      for j in neighbors_data: # Iterating through each neigbor data
        if (1-np.exp(-np.matmul(fact_matrix[i],fact_matrix[j].transpose()))): 
          Delta_12 = np.exp(-np.matmul(fact_matrix[i],fact_matrix[j].transpose()))/(1-np.exp(-np.matmul(fact_matrix[i],fact_matrix[j].transpose())))  
        else: 
          Delta_12 = 0 
        Delta_1 = (fact_matrix[j]*Delta_12) + Delta_1 #calculates the first term of the gradient function
        sum_nodes_v = fact_matrix[j] + sum_nodes_v
      Delta_2 = sum_nodes - fact_matrix[i] - sum_nodes_v 
      Delta_Main = Delta_1 - Delta_2 
      change = 0.1*sum(Delta_Main)/sum(fact_matrix[i])
      if change < 0.001: 
        percentage_change+=1
      fact_matrix[i] = fact_matrix[i] + (0.001*Delta_Main) 
      nn_vector = fact_matrix[i] < 0 
      fact_matrix[i][nn_vector] = 0 
    if percentage_change == len(G.nodes):
      break
  return fact_matrix

In [ ]:
#method1

fact_matrix_seed = bigclam_factor(300,fact_matrix, gt_communities,G)



In [ ]:
#method2
fact_matrix_neigh_bigclam = bigclam_factor(300,fact_matrix_neigh, gt_communities,G)


In [ ]:
#method3
fact_matrix_rand_bigclam = bigclam_factor(300,fact_matrix_rand, gt_communities,G)


In [ ]:
fact_matrix_rand_bigclam

#***Community assignment***

In [ ]:
##Define delta d
d = np.sqrt(-np.log(1-(10**-8)))


In [ ]:
#########This function takes factor matrix and d. This takes delta as the gradent and updates the community if F[u][c] >= delta 
def comm_membership(fact_matrix,d):
  comm= dict()
  for i in range(len(fact_matrix)):
    for j in range(len(fact_matrix[0])): 
      if j not in comm.keys(): 
        comm[j] = []
      if fact_matrix[i][j] >= d: # if the F[u][c] >= delta and updating the node in community
        comm[j].append(i)
  return comm


###This function writes teh cmmunities and respectives nodes into a file output
def write_dict_to_file(comm, outfile):
  f = open(outfile, 'w')
  for key, value in comm.items():
      f.write(str(key)+"::::>"+str(value)+'\n')
  f.close()



  


In [ ]:
#Method1 Community Assignment
community_seed = comm_membership(fact_matrix_seed,d)
write_dict_to_file(community_seed,'community_seed_output.txt')

#Method2 Community Assignment

community_neigh = comm_membership(fact_matrix_neigh_bigclam,d)
write_dict_to_file(community_neigh,'community_neigh_output.txt')

#Method3 Community Assignment

community_rand = comm_membership(fact_matrix_rand_bigclam,d)
write_dict_to_file(community_rand,'community_rand_output.txt')


In [ ]:
###Obtain Dictionary from of Cumminty and resective nodes
gt_dict = dict()
i=0
for line in gt_communities: 
    gt_dict[i] = line
    i+=1
print(gt_dict)

In [ ]:
##This function calculates the recall value. 
def eval_recall(gt_dict, community_seed):
  matching = {}
  collected = [] 
  for a in gt_dict: 
    for b in community_seed: 
      if a not in matching.keys(): 
        matching[a] = (-1,0)
      if  matching[a][1] < len(set(community_seed[a])&set(gt_dict[b]))/len(set(gt_dict[b])): 
        if b not in collected: 
          matching[a] = (b,len(set(community_seed[a])&set(gt_dict[b]))/len(set(gt_dict[b]))) # Updated precision score for the community associated
    collected.append(matching[a][0]) 
  recall_score = 0 
  for a in matching: 
    recall_score = recall_score + matching[a][1] 
  recall_score = recall_score/len(gt_dict.keys()) 
  return recall_score


##This function calculates the precision value for each of the methods 

def eval_precision(gt_dict, community_seed):
  matching = {}
  collected = [] 
  for a in gt_dict: 
    for b in community_seed: 
      if a not in matching.keys(): 
        matching[a] = (-1,0)
      if len(set(community_seed[b])) == 0: 
        continue
      else:
        if b not in collected: 
          matching[a] = (b,len(set(community_seed[a])&set(gt_dict[b]))/len(set(community_seed[b]))) # Updated precision score for the community associated
    collected.append(matching[a][0]) 
  precision_score = 0 
  for a in matching: 
    precision_score = precision_score + matching[a][1] 
  precision_score = precision_score/len(gt_dict.keys()) 
  return precision_score


In [ ]:
eval_recall(gt_dict,community_seed)

eval_recall(gt_dict,community_neigh)

eval_recall(gt_dict,community_rand)


In [ ]:
eval_recall(gt_dict,community_seed)


In [ ]:
plt.bar(["20 Percent Seed","Neighborhood","Randomly"],[eval_recall(gt_dict,community_seed),eval_recall(gt_dict,community_neigh),eval_recall(gt_dict,community_rand)], align='center', label="Recall Score")
plt.legend()
plt.ylabel('Recall')
plt.title('Recall vs. Matrix initializations')
plt.show()

In [ ]:
plt.bar(["20 Percent Seed","Neighborhood","Random"],[eval_precision(gt_dict,community_seed),eval_precision(gt_dict,community_neigh),eval_precision(gt_dict,community_rand)], align='center', label="Recall Score")
plt.legend()
plt.ylabel('Precision')
plt.title('Precision vs. Matrix initializations')
plt.show()

 **Interpretation**:Recall is generally used to assess False negatives and Precision is used to measure False Positives. Higher the value indicates less FN and FP respectively.
 
From the above **Two** Graphs, we can say that both 20 precent seed and Neighborhood methods have same precison and recall. However, Random has zero precision and recall. Hence we can conclude that 20 Percent seed and Neighborhood are better intitilization than random

